<a href="https://colab.research.google.com/github/MeGaNeKoS/Google_Colab/blob/master/GDrive_Direct_Download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GDrive Downloader 

In [ ]:
#@title Mount Google Drive
#@markdown To stream files we need to mount Google Drive.

from google.colab import drive
drive.mount('/content/drive')

## Start Download

###Torrent

In [ ]:
#@title Initialize torrent session
try:
  import libtorrent as lt
except:
  !pip install lbry-libtorrent
  import libtorrent as lt
from IPython.display import clear_output
clear_output()
import time
import random

SETTING = {
    'user_agent': 'qBittorrent/4.3.9',  # client spoof
    'peer_fingerprint': '-qB4390-',  # client spoof
    'active_seeds': 1000,
    'active_dht_limit': 1000,
    'allow_i2p_mixed': True,
    'dht_bootstrap_nodes': 'router.bittorrent.com:6881',
    'dht_announce_interval': 15,
    'min_announce_interval': 15,
    'connections_limit': 1000,
    'max_peerlist_size': 30000,
    'share_mode_target': 2,
    'max_web_seed_connections': 10,
    'allow_multiple_connections_per_ip': True,
    'announce_to_all_trackers': True,
    'announce_to_all_tiers': True,
}

ses = lt.session(SETTING)

ses.listen_on(random.randrange(10000, 50000), random.randrange(10000, 50000))

downloads = []

# Util function
def duration(num, msg="ETA... ", sep=":"):
    if int(num // 3600) > 0:
        msg += f"{int(num // 3600)}{sep}"
    if int(num // 60) > 0:
        msg += f"{int(num // 60) % 60}{sep}"
    msg += f'{int(num % 60)} sec'
    return msg

print("Ready to go")

In [ ]:
# @title Adding files

path = "/content/drive/Shared drives/Anime/Torrent"  # @param {type:"string"}
Torrent = False  # @param {type:"boolean"}
# @markdown Timeout (minute)
TIMEOUT = 1  # @param {type:"number"}
ANNOUNCE_INTERVAL = 15 # second
checking = []


if Torrent:
    print("Stop the code or cancel upload when you finish add the torrent files")

while True:
    if Torrent:
        from google.colab import files

        source = files.upload()
        try:
            params = {
                "save_path": path,
                "ti": lt.torrent_info(list(source.keys())[0]),
            }
            checking.append(ses.add_torrent(params))
        except Exception:
            break
    else:
        magnet_link = input("Enter Magnet Link Or Type E: ")
        if magnet_link.lower() == "e":
            break
        checking.append(
            lt.add_magnet_uri(ses, magnet_link, {"save_path": path})
        )

print("Checking for metadata")
timeout = 60 * TIMEOUT + time.time()
start = time.time()

while time.time() < timeout:
    for idx, torrent in enumerate(checking):
        try:
            if torrent.has_metadata():
                downloads.append(torrent)
                print(f"{torrent.name()} starting downloading")
                del checking[idx]
        except Exception:
            pass
        if ANNOUNCE_INTERVAL < time.time() - start:
            torrent.force_reannounce()
            torrent.force_dht_announce()
            start = time.time()
    if not checking:
        break
    time.sleep(.1)

for torrent in checking:
    print(f"{torrent.name()} has no metadata")
    ses.remove_torrent(torrent)


In [ ]:
# @title Display download progress
import threading
import time

import ipywidgets as widgets
from IPython.display import display

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]

print("You can stop, add files then resume again. this is only for displaying progress")
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()
            try:
                x = round(((s.total_done / ((s.progress * 100) / 100)) - s.total_done) / s.download_rate)
                if x > 359999:
                    filetime = duration(359999, 'Time left: ')
                else:
                    filetime = duration(x, 'Time left: ')
            except ZeroDivisionError:
                filetime = duration(359999, 'Time left: ')

            bar.description = " ".join(
                [
                    str(index),
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                    filetime,
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close()  # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)


def finish():
    from google.colab import output
    output.eval_js(
        'new Audio("https://drive.google.com/uc?export=download&id=1h-KhKaO-FNMsRKKYtiZ-AqNy8tr5mHcX").play()')


print("done")
t1 = threading.Thread(target=finish, args=(), daemon=False)
t1.start()
time.sleep(5)
if t1.is_alive():
    print(
        "\nYOU CAN IGNORE ANYTHING BELOW THIS.\nTHIS IS JUST TO FORCE STOP THE PROGRAM DUE AUDIO NOTIFICATION ERROR\n\n")
    raise SystemExit()  # sometime it got stuck when playing the notif audio.


In [ ]:
#@title delete a single file
index = 0 #@param {type:"number"}
del downloads[index]

In [ ]:
#@title delete file in range
start = 0 #@param {type:"number"}
end = 0 #@param {type:"number"}
del downloads[start:end]